<h1> einlesen der Daten </h1>

In [19]:
# import von Pandas, Betriebssystemfunktionen als Bibliotheken laden
import pandas as pd
import json
import os

<h2>Es folgen die Angaben der Speicherorte.</h2>

In [20]:
file_path = './data/'

# try to clean the raw data
# open the data json file and load it into the RAM
with open(file_path + 'data.json') as infile:
    # read the json structure
    d = json.load(infile)
    
    # d is a list of dict objects
    for book in d:
        # every book is a dict
        for key in book.keys():
            # replace the \n - new line character with a whitespace
            book[key] = book[key].replace('\n', ' ')
            # replace some words with whitespace
            book[key] = book[key].replace('[Fußnote]', ' ')
            book[key] = book[key].replace(';', ' ')
            
        book['year_dt'] = pd.to_datetime(book['year'], infer_datetime_format=True)
    
            

<h2> Daten in ein speichereffizientes Format bringen </h2>

In [21]:
# put all of the books into an pandas Dataframe
data = pd.DataFrame(d).set_index('number')

<h2>Mit dem folgenden Befehlen verschaffen wir uns einen Überblick über die eingeladenen Daten.</h2>

In [22]:
data  

,name,style,text,year,year_dt
number,,,,,
0001,Demetrius,Drama,Demetrius an der russischen Grenze Er ist vo...,1805,1805-01-01
0002,Der versöhnte Menschenfeind,Drama,Der versöhnte Menschenfeind - Fragment Gegend...,1790,1790-01-01
0003,Die Braut von Messina,Drama,Die Braut von Messina oder die feindlichen Brü...,1803,1803-01-01
0004,Die Huldigung der Künste,Drama,Ihrer Kaiserl. Hoheit der Frau Erbprinze...,1804,1804-01-01
0005,Die Jungfrau von Orleans,Drama,Die Jungfrau von Orleans Eine romantische Trag...,1801,1801-01-01
0006,Die Räuber,Drama,Die Räuber Ein Schauspiel Quae medicame...,1781,1781-01-01
0007,Die Verschwörung des Fiesco zu Genua,Drama,Die Verschwörung des Fiesco zu Genua Ein repub...,1783,1783-01-01
0008,Don Carlos,Drama,"Don Carlos, Infant von Spanien Ein dramatisch...",1788,1788-01-01
0009,Kabale und Liebe,Drama,Kabale und Liebe Ein bürgerliches Trauerspiel...,1784,1784-01-01


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 663 entries, 0001 to 0663
Data columns (total 5 columns):
name       663 non-null object
style      663 non-null object
text       663 non-null object
year       663 non-null object
year_dt    663 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 31.1+ KB


In [24]:
data.describe()

,name,style,text,year,year_dt
count,663,663,663,663,663
unique,395,7,662,30,30
top,an G Koerner,Brief,"Auch ich war in Arkadien geboren, Auch mir...",1795,1795-01-01 00:00:00
freq,21,478,2,78,78
first,NaN,NaN,NaN,NaN,1772-01-01 00:00:00
last,NaN,NaN,NaN,NaN,1805-01-01 00:00:00


<h1>NLP<h1>

<h2>Vorbereitung<h2>

Nun benötigen wir die spaCY-Bibliothek und das Modell für die deutsche Sprache.

Mehr zu spaCY: https://spacy.io

In [25]:
import spacy
from spacy.lang.de import German

nlp = spacy.load('de')

Das durch spaCY erzeugte NLP-Objekt ist sehr groß und benötigt viel Speicher. Um das etwas zu entschärfen,
werden nur die benötigten Daten, also die Anzahl der Wörter (Token), Adjektive, Substantive und Verben gespeichert. 
Dabei hilft defaultdict, weil hier alle Werte auf 0 gesetzt sind, das spart viele Fallunterscheidungen.

2 Do:
Die Worte selbst werden auch in ihrer lemmatisierten Form (also grammatikalisch auf die Grundform reduziert)
benötigt, um später Analysen damit durchführen zu können.

In [26]:
from datetime import datetime
from collections import defaultdict

startTime = datetime.now()


def calculate_the_word_types(data):
    nouns = defaultdict(lambda: 0)
    verbs = defaultdict(lambda: 0)
    adjectives = defaultdict(lambda: 0)
    adverbs = defaultdict(lambda: 0)

    for i, row in data.iterrows():
        doc = nlp(row["name"] + " " + row["text"])
        data.set_value(i, "nr_token", len(list(map(lambda x: x.text, 
                                         filter(lambda x: x.pos_ != 'PUNCT', doc)))))
    
        for a in map(lambda x: x.lemma_, filter(lambda x: x.pos_ == 'ADJ', doc)):
            adjectives[a] += 1
        data.set_value(i, "nr_adj", len(list(map(lambda x: x.text, 
                                         filter(lambda x: x.pos_ == 'ADJ', doc)))))  
    
        for n in map(lambda x: x.lemma_, filter(lambda x: (x.pos_ == 'NOUN') | (x.pos_ == 'PNOUN'), doc)):
            nouns[n] +=1
        data.set_value(i, "nr_noun", len(list(map(lambda x: x.text, 
                                         filter(lambda x: (x.pos_ == 'NOUN') | (x.pos_ == 'PNOUN'), doc)))))

        for v in map(lambda x: x.lemma_, filter(lambda x: (x.pos_ == 'AUX') | (x.pos_ == 'VERB'), doc)):
            verbs[v] += 1
        data.set_value(i, "nr_verb", len(list(map(lambda x: x.text, 
                                         filter(lambda x: (x.pos_ == 'AUX') | (x.pos_ == 'VERB'), doc)))))
                     
        for adv in map(lambda x: x.lemma_, filter(lambda x: x.pos_ == 'ADV', doc)):
            adverbs[adv] += 1
        data.set_value(i, "nr_adverb", len(list(map(lambda x: x.text, filter(lambda x: x.pos_ == 'ADV', doc)))))
    
    return data


###############################################
#
#
#          ACHTUNG DAS AUSFÜHREN DAUERT
#
#
#
#    KNAPP 12 MINUTEN - BITTE das PICKLE laden (11:19 min)
#
#               !!!!!!!!!!!!!!!
#
##############################################


data = calculate_the_word_types(data)
print(datetime.now() - startTime)

/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: FutureWarning: set_value is deprecated and will be removed in 

0:11:39.355851


<h2>calculate the percentages of wordtypes in the text</2>

In [27]:
data

,name,style,text,year,year_dt,nr_token,nr_adj,nr_noun,nr_verb,nr_adverb
number,,,,,,,,,,
0001,Demetrius,Drama,Demetrius an der russischen Grenze Er ist vo...,1805,1805-01-01,8504.0,601.0,1681.0,1481.0,515.0
0002,Der versöhnte Menschenfeind,Drama,Der versöhnte Menschenfeind - Fragment Gegend...,1790,1790-01-01,7132.0,586.0,1328.0,1284.0,454.0
0003,Die Braut von Messina,Drama,Die Braut von Messina oder die feindlichen Brü...,1803,1803-01-01,27192.0,2358.0,5121.0,3620.0,1360.0
0004,Die Huldigung der Künste,Drama,Ihrer Kaiserl. Hoheit der Frau Erbprinze...,1804,1804-01-01,1987.0,157.0,452.0,242.0,84.0
0005,Die Jungfrau von Orleans,Drama,Die Jungfrau von Orleans Eine romantische Trag...,1801,1801-01-01,32199.0,2308.0,6196.0,4800.0,1480.0
0006,Die Räuber,Drama,Die Räuber Ein Schauspiel Quae medicame...,1781,1781-01-01,42292.0,3063.0,7798.0,7434.0,3142.0
0007,Die Verschwörung des Fiesco zu Genua,Drama,Die Verschwörung des Fiesco zu Genua Ein repub...,1783,1783-01-01,31034.0,2408.0,6243.0,5375.0,1818.0
0008,Don Carlos,Drama,"Don Carlos, Infant von Spanien Ein dramatisch...",1788,1788-01-01,45866.0,2797.0,7635.0,6910.0,3171.0
0009,Kabale und Liebe,Drama,Kabale und Liebe Ein bürgerliches Trauerspiel...,1784,1784-01-01,31836.0,2356.0,6125.0,5475.0,2412.0


In [28]:
def calculate_percentages_of_word_types(data):

    data['per_verb'] = data['nr_verb']/(data['nr_token']/100)
    data['per_noun'] = data['nr_noun']/(data['nr_token']/100)
    data['per_adj'] = data['nr_adj']/(data['nr_token']/100)
    data['per_adverb'] = data['nr_adverb']/(data['nr_token']/100)
        
        
    return data



data = calculate_percentages_of_word_types(data)

<h2>using textstat for better information about the text difficulty</h2>

In [29]:
from textstat.textstat import textstat
# Python package to calculate statistics from text to determine readability,
# complexity and grade level of a particular corpus.
# https://github.com/shivam5992/textstat

def do_text_statistics(data):
    
    for i, row in data.iterrows():
        doc = row["name"] + " " + row["text"]
        
        textstat.syllable_count(doc, lang='de') # count the number of syllables
        textstat.lexicon_count(doc, True) # count the number of words - True = without punctuations
        textstat.sentence_count(doc) # count the number of sentences
        
        
    
        data.set_value(i, "flesch_reading_ease", textstat.flesch_reading_ease(doc))
        data.set_value(i, "smog_index", textstat.smog_index(doc))
        data.set_value(i, "flesch_kincaid_grade", textstat.flesch_kincaid_grade(doc))
        data.set_value(i, "coleman_liau_index", textstat.coleman_liau_index(doc))
        data.set_value(i, "automated_readability_index", textstat.automated_readability_index(doc))
        data.set_value(i, "dale_chall_readability_score", textstat.dale_chall_readability_score(doc))
        data.set_value(i, "difficult_words", textstat.difficult_words(doc))
        data.set_value(i, "linsear_write_formula", textstat.linsear_write_formula(doc))
        data.set_value(i, "gunning_fog", textstat.gunning_fog(doc))
        data.set_value(i, "text_standard", textstat.text_standard(doc, float_output=True))
        
    return data

# duration ca. 20-30 seconds
data = do_text_statistics(data)

/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/robinarthur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use

# Erklärung zu den 'Style'-Arten

D = Drama - Schauspiel - Lustspiele
G = Gedichte - Balladen,
E = Erzählung - Epik - prosaische Schriften,
H = historische Schriften,
P = Philosophische Schriften - Rezensionen,
N = Nacherzählungen, Übersetzungen,
B = Briefe an Freunde, Bekannte

In [30]:
data.T

number,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0654,0655,0656,0657,0658,0659,0660,0661,0662,0663
name,Demetrius,Der versöhnte Menschenfeind,Die Braut von Messina,Die Huldigung der Künste,Die Jungfrau von Orleans,Die Räuber,Die Verschwörung des Fiesco zu Genua,Don Carlos,Kabale und Liebe,Maria Stuart,...,an G Goeschen,an A Iffland,an F Frommann,an H Paulus,an F Niethammer,an L Frankh,an F Cotta,an F Stein,an W Goethe,an G Koerner
style,Drama,Drama,Drama,Drama,Drama,Drama,Drama,Drama,Drama,Drama,...,Brief,Brief,Brief,Brief,Brief,Brief,Brief,Brief,Brief,Brief
text,Demetrius an der russischen Grenze Er ist vo...,Der versöhnte Menschenfeind - Fragment Gegend...,Die Braut von Messina oder die feindlichen Brü...,Ihrer Kaiserl. Hoheit der Frau Erbprinze...,Die Jungfrau von Orleans Eine romantische Trag...,Die Räuber Ein Schauspiel Quae medicame...,Die Verschwörung des Fiesco zu Genua Ein repub...,"Don Carlos, Infant von Spanien Ein dramatisch...",Kabale und Liebe Ein bürgerliches Trauerspiel...,Maria Stuart Trauerspiel in fünf Aufzügen Eli...,...,[Weimar 25. Febr. (?) Freitag. 1805.] Hier üb...,"Weimar, 5. Januar [Mittwoch] 1805. Ich lebe au...",Weimar 3. April [Sonntag] 1805. Der Druck ist ...,"Weimar, 2. April [Sonnabend] 1805. Glauben Sie...",Weimar 2. April [Sonnabend] 1805. Mein langes ...,"Weimar 27. März [Sonntag] 1805. Ja, wohl ist e...","Weimar, 6. Januar [Donnerstag] 1805. Hubers To...",Weimar 13. April [Mittwoch] 1805. Der Schauspi...,14. Januar [Freitag] 1805. Es thut mir recht ...,Weimar 20. Jan. [Donnerstag] 1805. So wie das ...
year,1805,1790,1803,1804,1801,1781,1783,1788,1784,1800,...,1805,1805,1805,1805,1805,1805,1805,1805,1805,1805
year_dt,1805-01-01 00:00:00,1790-01-01 00:00:00,1803-01-01 00:00:00,1804-01-01 00:00:00,1801-01-01 00:00:00,1781-01-01 00:00:00,1783-01-01 00:00:00,1788-01-01 00:00:00,1784-01-01 00:00:00,1800-01-01 00:00:00,...,1805-01-01 00:00:00,1805-01-01 00:00:00,1805-01-01 00:00:00,1805-01-01 00:00:00,1805-01-01 00:00:00,1805-01-01 00:00:00,1805-01-01 00:00:00,1805-01-01 00:00:00,1805-01-01 00:00:00,1805-01-01 00:00:00
nr_token,8504,7132,27192,1987,32199,42292,31034,45866,31836,34830,...,872,816,280,263,412,484,2333,355,2527,1598
nr_adj,601,586,2358,157,2308,3063,2408,2797,2356,2258,...,57,79,28,23,31,42,234,26,213,139
nr_noun,1681,1328,5121,452,6196,7798,6243,7635,6125,6492,...,153,135,56,39,67,89,426,55,414,245
nr_verb,1481,1284,3620,242,4800,7434,5375,6910,5475,5171,...,115,125,36,39,64,77,351,58,407,268
nr_adverb,515,454,1360,84,1480,3142,1818,3171,2412,1661,...,60,66,21,35,38,51,191,31,263,153


In [31]:
# save the data into a pickle to save some time
import pickle

# -'./data/output_df_wth_all_nr.pickle' or
# -'./data/output_df_wth_all_nr_all_perc_reading.pickle' 

# save it  
#with open('./data/output_df_wth_all_nr_all_perc_reading.pickle', 'wb') as handle:
#    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# load it
with open('./data/output_df_wth_all_nr_all_perc_reading.pickle', 'rb') as handle:
    data = pickle.load(handle) 

In [18]:
data

,name,style,text,year,year_dt,nr_token,nr_adj,nr_noun,nr_verb,nr_adverb,...,flesch_reading_ease,smog_index,flesch_kincaid_grade,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard
number,,,,,,,,,,,,,,,,,,,,,
0001,Demetrius,Drama,Demetrius an der russischen Grenze Er ist vo...,1805,1805-01-01,8504.0,601.0,1681.0,1481.0,515.0,...,68.10,9.7,8.7,12.30,13.3,6.94,1233.0,6.250000,13.86,9.0
0002,Der versöhnte Menschenfeind,Drama,Der versöhnte Menschenfeind - Fragment Gegend...,1790,1790-01-01,7132.0,586.0,1328.0,1284.0,454.0,...,79.70,8.6,6.3,11.37,11.5,6.71,1026.0,4.900000,12.42,7.0
0003,Die Braut von Messina,Drama,Die Braut von Messina oder die feindlichen Brü...,1803,1803-01-01,27192.0,2358.0,5121.0,3620.0,1360.0,...,1.47,10.0,19.8,12.07,16.8,6.76,2753.0,10.800000,15.41,20.0
0004,Die Huldigung der Künste,Drama,Ihrer Kaiserl. Hoheit der Frau Erbprinze...,1804,1804-01-01,1987.0,157.0,452.0,242.0,84.0,...,8.20,9.2,19.3,11.79,17.1,7.75,300.0,6.714286,18.38,19.0
0005,Die Jungfrau von Orleans,Drama,Die Jungfrau von Orleans Eine romantische Trag...,1801,1801-01-01,32199.0,2308.0,6196.0,4800.0,1480.0,...,6.24,9.1,18.0,11.49,14.0,6.27,2783.0,65.000000,12.87,13.0
0006,Die Räuber,Drama,Die Räuber Ein Schauspiel Quae medicame...,1781,1781-01-01,42292.0,3063.0,7798.0,7434.0,3142.0,...,79.19,8.0,6.5,10.21,10.9,6.04,4148.0,5.888889,10.85,11.0
0007,Die Verschwörung des Fiesco zu Genua,Drama,Die Verschwörung des Fiesco zu Genua Ein repub...,1783,1783-01-01,31034.0,2408.0,6243.0,5375.0,1818.0,...,76.93,8.0,5.3,11.41,9.8,5.87,3303.0,16.750000,8.76,6.0
0008,Don Carlos,Drama,"Don Carlos, Infant von Spanien Ein dramatisch...",1788,1788-01-01,45866.0,2797.0,7635.0,6910.0,3171.0,...,3.56,7.7,17.0,10.44,10.9,5.72,3281.0,3.000000,9.91,10.0
0009,Kabale und Liebe,Drama,Kabale und Liebe Ein bürgerliches Trauerspiel...,1784,1784-01-01,31836.0,2356.0,6125.0,5475.0,2412.0,...,82.04,8.0,5.4,10.14,9.7,5.83,3037.0,3.214286,9.57,10.0
